In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export core
%nbdev_default_class_level 3

Cells will be exported to geomechy.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from geomechy.base import ItemDict

# Core
> Main program

## Material Manager

In [ ]:
%nbdev_export
class MaterialManager(list):
    
    def __init__(self, matProps):
        matType = matProps.type
        exec(f'from geomechy.materials import {matType} as material', globals())
        self.mat = material(matProps)
        self.iIter = -1

    def reset(self):
        self.iIter = -1

    def getStress(self, kinematic, iSam=-1):

        if iSam == -1:
            self.iIter += 1
            iSam = self.iIter

        self.mat.setIter(iSam)
        return self.mat.getStress(kinematic)

    def commitHistory(self):
        self.mat.commitHistory()

## Node Set Object

In [ ]:
%nbdev_export
class NodeSet(ItemDict):
    
    def getNodeCoords(self, nodeIds):
        return np.array(self.get(nodeIds))
    
    def readFromAbaqus(self, fileName):
        fin = open( fileName )
        
    def readFromJson(self, fileName):
        fin = open( fileName )

In [ ]:
nodes = NodeSet()
nodes.add( 1, [  0.,0.  ] ) #Attachement point of spring (can be positioned anywhere along the y-axis)
nodes.add( 2, [-10.,0.  ] ) #Left support
nodes.add( 3, [ 10.,0.  ] ) #Right support
nodes.add( 4, [ 0. ,0.5 ] ) #Loading point
nodes.get([1,2])

[[0.0, 0.0], [-10.0, 0.0]]

## Element Set Object

In [ ]:
class ElementSet(ItemDict):
    
    def __init__(self, nodes, props):
        ItemDict.__init__(self)
        self.nodes  = nodes
        self.props  = props
        self.groups = {}
    
    def __iter__(self):
        elements = []

        for groupName in self.iterGroupNames():
            for element in self.iterElementGroup(groupName):
                elements.append(element)

        return iter(elements)
    
    
    def getDofTypes(self):
        dofTypes = []
        
        for element in self:
            for dofType in element.dofTypes:
                if dofType not in dofTypes:
                    dofTypes.append(dofType)
        return dofTypes
    
    
    def readFromFile(self, fileName):
        fin = open(filename)
        
    
    def add(self, ID, modelName, elementNodes):  

        #Check if the model exists
        if not hasattr(self.props, modelName):
            RuntimeError('Missing properties for model ' + modelName)

        modelProps = getattr(self.props, modelName)

        
        #Check if the model has a type
        if not hasattr( modelProps, 'type' ):
            RuntimeError('Missing type for model ' + modelName)

        modelType = getattr(modelProps, 'type')

        
        #Load the element
        exec(f'from geomechy.elements import {modelType} as element',globals())

        
        #Create the element
        elem = element(elementNodes, modelProps)

        #Check if the node IDs are valid
        for nodeID in elem.getNodes():
            if not nodeID in self.nodes:
                raise RuntimeError('Node ID ' + str(nodeID) + ' does not exist')

        
        #Add the element to the element set
        ItemDict.add(self, ID, elem)

        #Add the element to the correct group
        self.addToGroup(modelName, ID)
    
    
    def addToGroup(self, modelType, ID):

        if not modelType in self.groups:
            self.groups[modelType] = [ID]
        else:
            self.groups[modelType].append( ID )

    def iterGroupNames(self):
        return self.groups
        
    def iterElementGroup (self, groupName):
        if groupName == "All":
            return iter(self)
        else:
            return iter(self.get(self.groups[groupName]))

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_materials.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
